In [45]:
import camelot 
# pip install camelot-py[cv], then install ghostscript by using conda, then pip install ghostscript for the python wrapper
from llama_index import Document, SummaryIndex
from llama_index import VectorStoreIndex, ServiceContext, LLMPredictor
from llama_index.retrievers import RecursiveRetriever
from llama_index.query_engine import PandasQueryEngine, RetrieverQueryEngine
from llama_index.schema import IndexNode
from llama_index.llms import OpenAI
from llama_index import download_loader
from pathlib import Path
from typing import List

In [46]:
"""Just don't use llama-hub, use download_loader instead"""
file_path = "./the_world_billionaires.pdf"
reader = download_loader("PyMuPDFReader")
loader = reader()
docs = loader.load_data(file_path=Path(file_path), metadata=True)

In [47]:
def get_tables(path: str, pages: List[int]):
    table_dfs = []
    for page in pages:
        table_list = camelot.read_pdf(path, pages=str(page))
        table_df = table_list[0].df
        table_df = (
            table_df.rename(columns=table_df.iloc[0])
            .drop(table_df.index[0])
            .reset_index(drop=True)
        )
        table_dfs.append(table_df)
    return table_dfs

In [48]:
table_dfs = get_tables(file_path, pages=[3, 25])

we create each query engine for each table

In [49]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.8)
service_context = ServiceContext.from_defaults(llm=llm)
df_query_engines = [PandasQueryEngine(table_df, service_context=service_context) for table_df in table_dfs]

In [50]:
response = df_query_engines[1].query("How many billionaires were there in 2009?")
print(str(response))

793


In [51]:
doc_nodes = service_context.node_parser.get_nodes_from_documents(docs)

the df_nodes is just a tuple with the description of the table(summary) and its index
After that, we want to map the df nodes to the list of df query engines created in the previous step.
the df node and the df engine is linked through the idx variable

In [52]:
summaries = [
    (
        "this node provides information about the world's richest billionaires in 2023"
    ),
    (
        "this node provides information on the numbner of billionaires and their combined networth from 2000 to 2023"
    )
]

df_nodes = [IndexNode(text=summary, index_id=f"pandas{idx}")
            for idx, summary in enumerate(summaries)]

df_id_query_engine_mapping = {
    f"pandas{idx}": df_query_engine for idx, df_query_engine in enumerate(df_query_engines)
}

In [56]:
summaries = [
    (
        "This node provides information about the world's richest billionaires"
        " in 2023"
    ),
    (
        "This node provides information on the number of billionaires and"
        " their combined net worth from 2000 to 2023."
    ),
]

df_nodes = [
    IndexNode(text=summary, index_id=f"pandas{idx}")
    for idx, summary in enumerate(summaries)
]

df_id_query_engine_mapping = {
    f"pandas{idx}": df_query_engine
    for idx, df_query_engine in enumerate(df_query_engines)
}

indexing our database, with all the texts plus the tables

In [57]:
vector_index = VectorStoreIndex(doc_nodes + df_nodes)
vector_retriever = vector_index.as_retriever(similarity_top_k=1)

TypeError: Object of type PosixPath is not JSON serializable